## 計算機イプシロンの求め方

計算機イプシロンとは「浮動小数点数において、「1より大きい最小の数」と1との差」([計算機イプシロン - Wikipedia](https://ja.wikipedia.org/wiki/%E8%A8%88%E7%AE%97%E6%A9%9F%E3%82%A4%E3%83%97%E3%82%B7%E3%83%AD%E3%83%B3))である．  
「1より大きい最小の数」は2進数で $1.00\dots01_{(2)}$ と表せるので，浮動小数点数の仮数部のビット数を $b$ として $2^{-b}$ である．

In [1]:
print("1 == 1 + 2^-52:", 1 == 1 + 2**-52)
print("1 == 1 + 2^-53:", 1 == 1 + 2**-53)

1 == 1 + 2^-52: False
1 == 1 + 2^-53: True


## floatのepsを求める

2のべきなので1との差を半半にしていけばよい

In [2]:
import math

def machine_eps(one):
    one_eps = one + one
    for b in range(10**3):
        if one == (one + one_eps) / 2:
            eps = one_eps - one
            return eps
        one_eps = (one + one_eps) / 2

eps = machine_eps(1)
print(f"{eps}, 2^{math.log2(eps)}")

2.220446049250313e-16, 2^-52.0


システム値とも一致する

In [3]:
import sys
sys.float_info.epsilon

2.220446049250313e-16

2のべきではない場合(そんなこと無いと思うが)，二分探索でも良いかもしれない．結果は同じ．

In [4]:
def machine_eps_bin(one):
    lower = one
    upper = one + one
    middle = (lower + upper) / 2
    while lower!=middle!=upper:
        one_eps = middle
        if one == middle:
            lower = middle
        else:
            upper = middle
        middle = (lower + upper) / 2
    eps = one_eps - one
    return eps

eps = machine_eps_bin(1)
print(f"{eps}, 2^{math.log2(eps)}")

2.220446049250313e-16, 2^-52.0


## 同じようで違う定義

「$1+x\neq 1$ となる最小の $x$」と同値であるかのような定義や説明がされることがあるが，実際は加算結果が桁落ちした際の丸め処理によっては一致しない．  
実際，Pythonでこの定義に基づいて計算を行うと正しい値の半分になる．  
参考：[計算機イプシロンのこと - 再帰の反復blog](https://lemniscus.hatenablog.com/entry/20090816/1250441897)

In [5]:
def machine_eps_diff(one):
    lower = one * 0
    upper = one
    middle = (lower + upper) / 2
    while lower!=middle!=upper:
        eps = middle
        if one == one + eps:
            lower = middle
        else:
            upper = middle
        middle = (lower + upper) / 2
    return eps

eps = machine_eps_diff(1)
print(f"{eps}, 2^{math.log2(eps)}")

1.1102230246251568e-16, 2^-53.0


## その他の精度の浮動小数点数のeps

numpy を使って半精度から4倍精度まで確かめる

In [6]:
import numpy as np

In [7]:
print(f"{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps(np.array([1], dtype=t))
    print(f"{str(t().dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

type    :            eps
float16 : 9.76562500e-04, 2^-10.0
float32 : 1.19209290e-07, 2^-23.0
float64 : 2.22044605e-16, 2^-52.0
float128: 1.08420217e-19, 2^-63.0


システム値とも一致する

In [8]:
print(f"{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    print(f"{str(t().dtype):<8}: {np.finfo(t).eps:.8e}")

type    :            eps
float16 : 9.76562500e-04
float32 : 1.19209290e-07
float64 : 2.22044605e-16
float128: 1.08420217e-19


スカラーを取り出して計算すると4倍精度以外は`float`扱いになる？

In [9]:
print(f"{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps(np.array([1], dtype=t)[0])
    print(f"{str(t().__class__.__name__):<8}: {eps:.8e}, 2^{math.log2(eps)}")

type    :            eps
float16 : 2.22044605e-16, 2^-52.0
float32 : 2.22044605e-16, 2^-52.0
float64 : 2.22044605e-16, 2^-52.0
float128: 1.08420217e-19, 2^-63.0


2分探索した場合も同様

In [10]:
print("np.array([1], dtype=type)")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps_bin(np.array([1], dtype=t))
    print(f"\t{str(t().dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("np.array([1], dtype=type)[0]")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps_bin(np.array([1], dtype=t)[0])
    print(f"\t{str(t().__class__.__name__):<8}: {eps:.8e}, 2^{math.log2(eps)}")

np.array([1], dtype=type)
	type    :            eps
	float16 : 9.76562500e-04, 2^-10.0
	float32 : 1.19209290e-07, 2^-23.0
	float64 : 2.22044605e-16, 2^-52.0
	float128: 1.08420217e-19, 2^-63.0
np.array([1], dtype=type)[0]
	type    :            eps
	float16 : 2.22044605e-16, 2^-52.0
	float32 : 2.22044605e-16, 2^-52.0
	float64 : 2.22044605e-16, 2^-52.0
	float128: 1.08420217e-19, 2^-63.0


floatと同様に，1との差で計算すると正しい値の半分になる．

In [11]:
print("np.array([1], dtype=type)")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps_diff(np.array([1], dtype=t))
    print(f"\t{str(t().dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("np.array([1], dtype=type)[0]")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    eps = machine_eps_diff(np.array([1], dtype=t)[0])
    print(f"\t{str(t().__class__.__name__):<8}: {eps:.8e}, 2^{math.log2(eps)}")

np.array([1], dtype=type)
	type    :            eps
	float16 : 4.88758087e-04, 2^-10.998591805607191
	float32 : 5.96046519e-08, 2^-23.99999982801736
	float64 : 1.11022302e-16, 2^-53.0
	float128: 5.42101086e-20, 2^-64.0
np.array([1], dtype=type)[0]
	type    :            eps
	float16 : 1.11022302e-16, 2^-53.0
	float32 : 1.11022302e-16, 2^-53.0
	float64 : 1.11022302e-16, 2^-53.0
	float128: 5.42101086e-20, 2^-64.0


## 相対誤差の評価

相対誤差を評価するとき，基準が2のベキならepsと一致するが，それ以外の場合はズレる

In [12]:
def rel_machine_eps(one, base):
    lower = base
    upper = base + base
    middle = (lower + upper) / 2
    while lower!=middle!=upper:
        base_eps = middle
        if (base - middle) / base == 0:
            lower = middle
        else:
            upper = middle
        middle = (lower + upper) / 2
    eps = (base_eps - base) / base
    return eps

print("base=2^-1")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    one = np.array([1], dtype=t)
    base = one / 2
    eps = rel_machine_eps(one, base)
    print(f"\t{str(base.dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("base=2^-4")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    one = np.array([1], dtype=t)
    base = one / (1 << 4)
    eps = rel_machine_eps(one, base)
    print(f"\t{str(base.dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("base=2^4")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    one = np.array([1], dtype=t)
    base = one * (1 << 4)
    eps = rel_machine_eps(one, base)
    print(f"\t{str(base.dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("base=5")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    one = np.array([1], dtype=t)
    base = one * 5
    eps = rel_machine_eps(one, base)
    print(f"\t{str(base.dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

print("base=1234")
print(f"\t{'type':<8}: {'eps':>14}")
for t in [np.float16, np.float32, np.float64, np.float128]:
    one = np.array([1], dtype=t)
    base = one * 1234
    eps = rel_machine_eps(one, base)
    print(f"\t{str(base.dtype):<8}: {eps[0]:.8e}, 2^{math.log2(eps)}")

base=2^-1
	type    :            eps
	float16 : 9.76562500e-04, 2^-10.0
	float32 : 1.19209290e-07, 2^-23.0
	float64 : 2.22044605e-16, 2^-52.0
	float128: 1.08420217e-19, 2^-63.0
base=2^-4
	type    :            eps
	float16 : 9.76562500e-04, 2^-10.0
	float32 : 1.19209290e-07, 2^-23.0
	float64 : 2.22044605e-16, 2^-52.0
	float128: 1.08420217e-19, 2^-63.0
base=2^4
	type    :            eps
	float16 : 9.76562500e-04, 2^-10.0
	float32 : 1.19209290e-07, 2^-23.0
	float64 : 2.22044605e-16, 2^-52.0
	float128: 1.08420217e-19, 2^-63.0
base=5
	type    :            eps
	float16 : 7.81059265e-04, 2^-10.322280358358991
	float32 : 9.53674331e-08, 2^-23.321928073389532
	float64 : 1.77635684e-16, 2^-52.32192809488736
	float128: 8.67361738e-20, 2^-63.32192809488736
base=1234
	type    :            eps
	float32 : 9.89224560e-08, 2^-23.269126700311347
	float32 : 9.89224560e-08, 2^-23.269126700311347
	float64 : 1.84257436e-16, 2^-52.269126679149416
	float128: 8.99694509e-20, 2^-63.269126679149416


### おまけ

decimal でも計算できるが，浮動小数ではないので特に意味はない

In [13]:
from decimal import Decimal
eps = machine_eps_bin(Decimal(1))
print(f"{str(eps.__class__.__name__):<8}: {eps:.8e}, 2^{math.log2(eps)}")

Decimal : 1.00000000e-27, 2^-89.69205856195879
